#Implementing the LSTM

In [ ]:
import numpy as np
import re
np.random.seed(42)

In [ ]:
data = open("sh.txt").read().lower()

In [ ]:
data = data.rstrip()
data = data[:5000]
print(len(data))

5000


In [ ]:
l = [",",".","?","|","(",")"]

for i in l:
  data = data.replace(i,"")

data = re.sub("(\n+)"," ",data)
data = data.split(" ")

chars = set(data)
vocab_size = len(chars)

ch2id = {c : i for i,c in enumerate(chars)}
id2ch = {i:c for i,c in enumerate(chars)}

print(vocab_size)
print(ch2id)

442
{'': 0, "summer's": 1, 'refigured': 2, 'answer': 3, 'spend': 4, 'delights': 5, 'place': 6, 'feeble': 7, 'world': 8, 'deep': 9, "th'": 10, 'churl': 11, 'beauteous': 12, 'image': 13, 'back': 14, 'due': 15, 'sum': 16, 'play': 17, 'burning': 18, 'fair': 19, 'eyes': 20, 'son': 21, 'my': 22, 'ornament': 23, 'self': 24, 'all-eating': 25, 'each': 26, 'his': 27, 'leese': 28, 'serving': 29, 'climbed': 30, 'an': 31, 'very': 32, 'or': 33, 'frost': 34, 'lends': 35, 'in': 36, 'pay': 37, 'die': 38, 'womb': 39, 'only': 40, 'confounds': 41, 'by': 42, 'child': 43, 'ten': 44, 'posterity': 45, 'proud': 46, "consum'st": 47, 'flame': 48, 'making': 49, 'middle': 50, 'beauty': 51, 'and': 52, 'song': 53, 'stop': 54, 'fond': 55, 'windows': 56, "winter's": 57, 'way:': 58, 'gladly': 59, 'strikes': 60, 'bear': 61, 'highmost': 62, 'days;': 63, 'made': 64, 'lies': 65, 'how': 66, 'eye': 67, 'time': 68, "excuse'": 69, 'lovely': 70, 'usurer': 71, 'trenches': 72, 'see': 73, 'praise': 74, 'the': 75, 'youth': 76, 'hel

In [ ]:
len(data) // 20

44

In [ ]:
seqlen = 20
num_batches = 44

data_trimmed = data[:num_batches * seqlen]
batches = []
for j in range(0,len(data_trimmed) - seqlen, seqlen):
  batch = []
  x_batch = [ch2id[ch] for ch in data_trimmedj+1:j+seqlen+1]]
  batch.append(x_batch)
  batch.append(y_batch)
  batches.append(batch)

In [ ]:
len(batches)

43

In [ ]:
def sigmoid(x, derivative=False):
  x_safe = x + 1e-12
  f = 1 / (1 + np.exp(-x_safe))
  
  if derivative: # Return the derivative of the function evaluated at x
      return f * (1 - f)
  else: # Return the forward pass of the function at x
      return f

def tanh(x, derivative=False):
  x_safe = x + 1e-12
  f = (np.exp(x_safe)-np.exp(-x_safe))/(np.exp(x_safe)+np.exp(-x_safe))
  
  if derivative: # Return the derivative of the function evaluated at x
      return 1-f**2
  else: # Return the forward pass of the function at x
      return f

def softmax(x, derivative=False):
  x_safe = x + 1e-12
  f = np.exp(x_safe) / np.sum(np.exp(x_safe))
  
  if derivative: # Return the derivative of the function evaluated at x
      pass # We will not need this one
  else: # Return the forward pass of the function at x
      return f

# def softmax(x):
#   e_x = np.exp(x - np.max(x))
#   return e_x / e_x.sum(axis=0)



# Implementing the LSTM 

In [ ]:
# def init_orthogonal(param):
#   if param.ndim < 2:
#         raise ValueError("Only parameters with 2 or more dimensions are supported.")

#   rows, cols = param.shape
  
#   new_param = np.random.randn(rows, cols)
  
#   if rows < cols:
#       new_param = new_param.T
#   q, r = np.linalg.qr(new_param)
  

#   d = np.diag(r, 0)
#   ph = np.sign(d)
#   q *= ph

#   if rows < cols:
#       q = q.T
  
#   new_param = q
    
#   return new_param

In [ ]:
hidden_size = 64

z_size = hidden_size + vocab_size

def init_lstm(hidden_size, vocab_size, z_size):
  W_f = np.zeros((hidden_size, hidden_size+vocab_size))
  b_f = np.zeros((hidden_size, 1))
  W_i = np.zeros((hidden_size, hidden_size+vocab_size))
  b_i = np.zeros((hidden_size,1))
  W_g = np.zeros((hidden_size, hidden_size+vocab_size))
  b_g = np.zeros((hidden_size, 1))
  W_o = np.zeros((hidden_size, hidden_size+vocab_size))
  b_o = np.zeros((hidden_size, 1))
  W_v = np.zeros((vocab_size,hidden_size ))
  b_v = np.zeros((vocab_size, 1))

  W_f = init_orthogonal(W_f)
  W_i = init_orthogonal(W_i)
  W_g = init_orthogonal(W_g)
  W_o = init_orthogonal(W_o)
  W_v = init_orthogonal(W_v)

  return W_f, W_i, W_g, W_o, W_v, b_f, b_i, b_g, b_o, b_v

params = init_lstm(hidden_size=hidden_size, vocab_size=vocab_size, z_size=z_size)


In [ ]:
def update_parameters(params, grads, lr=1e-3):
  # Take a step
  for param, grad in zip(params, grads):
      param -= (lr * grad)

  return params

In [ ]:
def clip_gradient_norm(grads, max_norm=0.25):
  max_norm = float(max_norm)
  total_norm = 0
  
  # Calculate the L2 norm squared for each gradient and add them to the total norm
  for grad in grads:
      grad_norm = np.sum(np.power(grad, 2))
      total_norm += grad_norm
  
  total_norm = np.sqrt(total_norm)
  
  # Calculate clipping coeficient
  clip_coef = max_norm / (total_norm + 1e-6)
  
  # If the total norm is larger than the maximum allowable norm, then clip the gradient
  if clip_coef < 1:
      for grad in grads:
          grad *= clip_coef
  
  return grads

In [ ]:
params = init_lstm(hidden_size=hidden_size, vocab_size=vocab_size, z_size=z_size)

def forward(inputs, h_prev, C_prev, p):
  assert h_prev.shape == (hidden_size, 1)
  assert C_prev.shape == (hidden_size, 1)

  # First we unpack our parameters
  W_f, W_i, W_g, W_o, W_v, b_f, b_i, b_g, b_o, b_v = p
  
  # Save a list of computations for each of the components in the LSTM
  x_s, z_s, f_s, i_s,  = [], [] ,[], []
  g_s, C_s, o_s, h_s = [], [] ,[], []
  v_s, outputs =  [], [] 
  
  # Append the initial cell and hidden state to their respective lists
  h_s.append(h_prev)
  C_s.append(C_prev)

  for x in inputs:
    z = np.row_stack((h_prev,x))
    z_s.append(z)

    f = sigmoid(np.dot(W_f, z) + b_f)
    f_s.append(f)

    i = sigmoid(np.dot(W_i,z) + b_i)
    i_s.append(i)

    g = tanh(np.dot(W_g, z) + b_g)
    g_s.append(g)

    C_prev = C_prev * f + g * i
    C_s.append(C_prev)

    o = sigmoid(np.dot(W_o,z) + b_o)
    o_s.append(o)

    h_prev = o * tanh(C_prev)
    h_s.append(h_prev)

    # Calculate logits
    v = np.dot(W_v, h_prev) + b_v
    v_s.append(v)
    
    # Calculate softmax
    output = softmax(v)
    outputs.append(output)

  return z_s, f_s, i_s, g_s, C_s, o_s, h_s, v_s, outputs


In [ ]:
def backward(z, f, i, g, C, o, h, v, outputs, targets, p = params):

    # Unpack parameters
    W_f, W_i, W_g, W_o, W_v, b_f, b_i, b_g, b_o, b_v = p

    # Initialize gradients as zero
    W_f_d = np.zeros_like(W_f)
    b_f_d = np.zeros_like(b_f)

    W_i_d = np.zeros_like(W_i)
    b_i_d = np.zeros_like(b_i)

    W_g_d = np.zeros_like(W_g)
    b_g_d = np.zeros_like(b_g)

    W_o_d = np.zeros_like(W_o)
    b_o_d = np.zeros_like(b_o)

    W_v_d = np.zeros_like(W_v)
    b_v_d = np.zeros_like(b_v)
    
    # Set the next cell and hidden state equal to zero
    dh_next = np.zeros_like(h[0])
    dC_next = np.zeros_like(C[0])
        
    # Track loss
    loss = 0
    
    for t in reversed(range(len(outputs))):
        
        # Compute the cross entropy
        loss += -np.mean(np.log(outputs[t]) * targets[t])
        # Get the previous hidden cell state
        C_prev= C[t-1]
        
        # Compute the derivative of the relation of the hidden-state to the output gate
        dv = np.copy(outputs[t])
        dv[np.argmax(targets[t])] -= 1

        # Update the gradient of the relation of the hidden-state to the output gate
        W_v_d += np.dot(dv, h[t].T)
        b_v_d += dv

        # Compute the derivative of the hidden state and output gate
        dh = np.dot(W_v.T, dv)        
        dh += dh_next
        do = dh * tanh(C[t])
        do = sigmoid(o[t], derivative=True)*do
        
        # Update the gradients with respect to the output gate
        W_o_d += np.dot(do, z[t].T)
        b_o_d += do

        # Compute the derivative of the cell state and candidate g
        dC = np.copy(dC_next)
        dC += dh * o[t] * tanh(tanh(C[t]), derivative=True)
        dg = dC * i[t]
        dg = tanh(g[t], derivative=True) * dg
        
        # Update the gradients with respect to the candidate
        W_g_d += np.dot(dg, z[t].T)
        b_g_d += dg

        # Compute the derivative of the input gate and update its gradients
        di = dC * g[t]
        di = sigmoid(i[t], True) * di
        W_i_d += np.dot(di, z[t].T)
        b_i_d += di

        # Compute the derivative of the forget gate and update its gradients
        df = dC * C_prev
        df = sigmoid(f[t]) * df
        W_f_d += np.dot(df, z[t].T)
        b_f_d += df

        # Compute the derivative of the input and update the gradients of the previous hidden and cell state
        dz = (np.dot(W_f.T, df)
             + np.dot(W_i.T, di)
             + np.dot(W_g.T, dg)
             + np.dot(W_o.T, do))
        dh_prev = dz[:hidden_size, :]
        dC_prev = f[t] * dC
        
    grads= W_f_d, W_i_d, W_g_d, W_o_d, W_v_d, b_f_d, b_i_d, b_g_d, b_o_d, b_v_d
    
    # Clip gradients
    grads = clip_gradient_norm(grads)
    
    return loss, grads

In [ ]:
def one_hot_encode(idx,vocab_size):
  one_hot = np.zeros((vocab_size,1))
  one_hot[idx] = 1.0
  return one_hot

def one_hot_sequence(sequence, vocab_size):
  # here sequence is numbers of ids
  encoding = np.array([one_hot_encode(word,vocab_size) for word in sequence])
  encoding = encoding.reshape(encoding.shape[0], encoding.shape[1],1)
  return encoding


In [ ]:
num_epochs = 1000
z_size = hidden_size + vocab_size
params = init_lstm(hidden_size=hidden_size, vocab_size=vocab_size, z_size=z_size)

hidden_state = np.zeros((hidden_size,1))
training_loss= []

for i in range(num_epochs):
  epoch_training_loss = 0
  h = np.zeros((hidden_size,1))
  c = np.zeros((hidden_size,1))  
  for batch in batches:
    x_batch = batch[0]
    y_batch = batch[1]
    
    x_seq = one_hot_sequence(x_batch, vocab_size)
    y_seq = one_hot_sequence(y_batch, vocab_size)


    z_s, f_s, i_s, g_s, C_s, o_s, h_s, v_s, outputs = forward(x_seq, h, c, params)
    loss, grads = backward(z_s, f_s, i_s, g_s, C_s, o_s, h_s, v_s, outputs, y_seq, params)
    # print("Params before : " ,params[0])
    params = update_parameters(params = params, grads = grads, lr = 0.001)
    # print("Params after : ", params[0])
  epoch_training_loss += loss
  training_loss.append(epoch_training_loss)
  print(f'Epoch {i}, training loss: {training_loss[-1]}')

Epoch 0, training loss: 0.27556917303528033
Epoch 1, training loss: 0.27555932547069745
Epoch 2, training loss: 0.2755494799730793
Epoch 3, training loss: 0.27553963654323116
Epoch 4, training loss: 0.2755297951819593
Epoch 5, training loss: 0.275519955890071
Epoch 6, training loss: 0.2755101186683745
Epoch 7, training loss: 0.275500283517679
Epoch 8, training loss: 0.2754904504387949
Epoch 9, training loss: 0.27548061943253316
Epoch 10, training loss: 0.2754707904997061
Epoch 11, training loss: 0.275460963641127
Epoch 12, training loss: 0.27545113885760986
Epoch 13, training loss: 0.27544131614997
Epoch 14, training loss: 0.2754314955190233
Epoch 15, training loss: 0.275421676965587
Epoch 16, training loss: 0.2754118604904791
Epoch 17, training loss: 0.2754020460945186
Epoch 18, training loss: 0.27539223377852545
Epoch 19, training loss: 0.27538242354332054
Epoch 20, training loss: 0.2753726153897258
Epoch 21, training loss: 0.275362809318564
Epoch 22, training loss: 0.275353005330658

In [ ]:
sample_size = 20

def sample(params,ch2id,id2ch,vocab_size, hidden_size, sample_size):
  W_f, W_i, W_g, W_o, W_v, b_f, b_i, b_g, b_o, b_v = params

  sample_string = ""

  h = np.zeros((hidden_size,1))
  c = np.zeros((hidden_size,1))
  h_prev  = h
  C_prev = c

  x = np.zeros((vocab_size,1))

  for t in range(sample_size):
    z = np.row_stack((h_prev,x))
    f = sigmoid(np.dot(W_f, z) + b_f)
    i = sigmoid(np.dot(W_i,z) + b_i)
    g = tanh(np.dot(W_g, z) + b_g)
    C_prev = C_prev * f + i * g
    o = sigmoid(np.dot(W_o,z) + b_o)
    h_prev = o * tanh(C_prev)
    v = np.dot(W_v, h_prev) + b_v  
    output = softmax(v)

    # print(t)

    idx = np.random.choice(range(vocab_size), p = np.ravel(output))
    # idx = np.argmax(output)
    x = np.zeros((vocab_size,1))
    x[idx] = 1

    sample_string += id2ch[idx] + " "
    # print(sample_string)

  return sample_string


s = sample(params,ch2id,id2ch,vocab_size, hidden_size, sample_size)
print(s)




gladly hear'st strong time bud attending face cruel: self-killed: which see never-resting light where sweetly make gaze grave prime singleness 


In [ ]:
s

"gazed consum'st heir head willing thine climbed homage tillage prime another where under bounteous highmost through shame loan; widow's left "